In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload


import sys, os, inspect
import sys
sys.path.append('/Users/emigardiner/VICO/pjams-ionization/pjams/')

from zeusmp_snapshot_reader import read_zeusmp_snapshot
from zeusmp_snapshot_reader import ScaleFactors
from snapshot import snapshot 
from basic_snapshot import basic_snapshot, VICO_loc, FREQS
import plot
import flux

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib.colors import TwoSlopeNorm
from matplotlib.lines import Line2D

from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as DisplayImage

In [ ]:
# INPUTS
freqs = np.array([.01, .05, .1, .5, 1, 5.3, 23, 43, 100, 230 ]) # GHz
freqs *= 10**9 # Hz
scales = np.array([1000, 2000, 4000, 8000, 16000, 25000])

# r_kpc = 1
# heights_and_scales = np.load(VICO_loc+'/Data/heights_and_scales.npz')
# # scales = heights_and_scales['scales'] # AU
# heights = heights_and_scales['heights'] # AU  

colors = plot.COLORS

In [ ]:
nums = np.array([4, 9, 21, 39, 54, 68, 94])
years = np.array([4000, 9000, 21000, 39000, 54000, 68000, 94000])
# years = np.array(['4,000 yrs', '9,000 yrs', '21,000 yrs', '39,000 yrs', '54,000 yrs', '68,000 yrs', '94,000 yrs'])
masss = np.array([r'1.4 M$_\odot$', r'2 M$_\odot$', r'4 M$_\odot$',
                   r'8 M$_\odot$', r'12 M$_\odot$', r'16 M$_\odot$',
                   r'24 M$_\odot$'])

In [ ]:
snaps = np.empty_like(nums, dtype=snapshot)
for ii, num in enumerate(nums):
    snaps[ii] = basic_snapshot(snap=num, name = ('Snap%03d_n' % num))
    snaps[ii].load_variabilities()

In [ ]:
ylabel=r'Flux, $S_\nu$ (mJy)'
labels = np.array(['1,000 au', '2,000 au', '4,000 au', '8,000 au', 
                   '16,000 au', '25,000 au'])
dimcolors= np.array(['#9467bd', '#677cbc', '#67bca7', '#bcbc67',  '#bc9167', '#bc6767' ])
colors= np.array(['#9966cc', '#667fcc', '#66ccb3', '#b3cc66', '#cc9966', '#cc6666' ])
# ylim0 = (10**-1.5, 10**3.5)

# ff_arr = [5, 9]
fig, axs = plot.figax(figsize = (11,4), ncols=2,
    xlabel='Years', sharex=True, xscale='linear',)
# for ax in axs[:,0]:
axs[0].set_ylabel(ylabel)

avg_str = np.array(['averages', 'averages_const'])
std_str = np.array(['stdevs', 'stdevs_ratio'])

for aa, ax in enumerate(axs):
    xx = years
    y1 = np.zeros((len(scales), len(snaps)))
    err1 = np.zeros_like(y1)
    y2 = np.zeros((len(scales), len(snaps)))
    err2 = np.zeros_like(y2)
    for ss, snap in enumerate(snaps):
        # print(ss)
        y1[:,ss]   = 10**snap.Variabilities[avg_str[aa]][5,1:]
        err1[:,ss] = 10**snap.Variabilities[std_str[aa]][5,1:]
        y2[:,ss]   = 10**snap.Variabilities[avg_str[aa]][9,1:]
        err2[:,ss] = 10**snap.Variabilities[std_str[aa]][9,1:]

    fflabels = ['5.3 GHz', '230 GHz']

    for ii, scale in enumerate(scales):
        label = labels[ii] #if aa==1 else None
        ax.errorbar(xx, y1[ii], yerr=err1[ii], color=colors[ii],
                         capsize=4, label=label, alpha=0.75, linewidth=2, linestyle='-')
        ax.errorbar(xx, y2[ii], yerr=err2[ii], color=colors[ii],
                         capsize=4, label=None, alpha=0.75, linewidth=2, linestyle='--')
    axtext = ['no cooling', 'with cooling']
    for ii, ax in enumerate(axs):
        ax.text(0.99,0.95, axtext[ii], transform=ax.transAxes, weight='bold', horizontalalignment='right')
# axs[0,0].set_ylim(ylim0)
# axs[0,1].sharey(axs[0,0])
axs[0].legend(loc='lower right')
legend_elements = [Line2D([0], [0], color='k', lw=2, linestyle='-', label='5.3 GHz'),
                   Line2D([0], [0], color='k', lw=2, linestyle='--', label='230 GHz')]
axs[1].legend(handles=legend_elements, bbox_to_anchor=(1, 0.95), loc='upper right')


fig.tight_layout()
fig.savefig(VICO_loc+'/figures/flux_vs_time.png', dpi=300)
